In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('localhost', 27017)
collection = client['companies'].tableauproject

In [2]:
# hacemos una consulta para que nos dé toda la colección y lo convertimos en df
data = pd.DataFrame(list(collection.find()))
data.head()

,_id,category_code,city,country_code,founded_year,geopoint,latitude,longitude,name,number_of_employees,type
0,5cd453a1e74b56f50369c6e3,web,New York,USA,2005,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431,Wetpaint,47,startup
1,5cd453a1e74b56f50369c6e4,games_video,San Francisco,USA,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,Kyte,40,startup
2,5cd453a1e74b56f50369c6e5,web,Pleasanton,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,startup
3,5cd453a1e74b56f50369c6e6,games_video,Los Angeles,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,startup
4,5cd453a1e74b56f50369c6e7,games_video,San Bruno,USA,2005,"{'type': 'Point', 'coordinates': [-122.426804,...",37.627971,-122.426804,YouTube,0,startup


In [4]:
# función para sacar el número de empresas que hay en un radio específico de un geopoint concreto
def get_near_companies(geopoint, max_distance):
    nearcomps = pd.DataFrame(list(collection.find(
        {'geopoint': { '$near': {'$geometry': geopoint, '$maxDistance': max_distance}}}, {'name'})))
    return len(nearcomps)

In [5]:
# creamos columna nueva con el número de empresas cerca de cada empresa
distances = [200, 500, 1000, 2000]
for e in distances:
    data['companies{}'.format(e)] = data['geopoint'].apply(lambda x: get_near_companies(x, e))

In [6]:
# función para sacar el número de empleados que hay en un radio específico de un geopoint concreto
def get_near_employees(geopoint, max_distance):
    nearemps = pd.DataFrame(list(collection.find(
        {'geopoint': { '$near': {'$geometry': geopoint, '$maxDistance': max_distance}}})))
    return sum(nearemps['number_of_employees'])

In [7]:
# creamos columna nueva con el número de empleados cerca de cada empresa
for e in distances:
    data['employees{}'.format(e)] = data['geopoint'].apply(lambda x: get_near_employees(x, e))

In [8]:
data.head()

,_id,category_code,city,country_code,founded_year,geopoint,latitude,longitude,name,number_of_employees,type,companies200,companies500,companies1000,companies2000,employees200,employees500,employees1000,employees2000
0,5cd453a1e74b56f50369c6e3,web,New York,USA,2005,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431,Wetpaint,47,startup,1,2,9,28,47,53,230,431
1,5cd453a1e74b56f50369c6e4,games_video,San Francisco,USA,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,Kyte,40,startup,2,5,23,85,55,71,252,2178
2,5cd453a1e74b56f50369c6e5,web,Pleasanton,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,startup,1,1,2,2,22,22,1622,1622
3,5cd453a1e74b56f50369c6e6,games_video,Los Angeles,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,startup,1,2,2,8,35,70,70,167
4,5cd453a1e74b56f50369c6e7,games_video,San Bruno,USA,2005,"{'type': 'Point', 'coordinates': [-122.426804,...",37.627971,-122.426804,YouTube,0,startup,1,1,1,1,0,0,0,0


In [9]:
data.to_csv('tableau_project_pt2.csv')

In [11]:
# TO DO: subir el archivo a Drive desde el programa